# Learning the direction pointing to the mirror
**Goal**: given an image, learn which direction to turn that will bring the robot closer to the mirror object

In [24]:
import tensorflow as tf
import numpy
from MirrorAI.dataset.directional import sample as make_samples

## Define constants

In [286]:
image_width = 16
image_height = 1
vision_range = image_width * image_height
image_size = image_width * image_height
category_min = 1
category_max = 2
category_count = category_max - category_min + 1
label_type_count = 3  # [-1, 0, 1]

segment_count = {
    "training": 8000,
    "test": 1300,
    "validation": 2000
    }
sample_count = {
    "training": segment_count['training'] * vision_range,
    "test": segment_count['test'] * vision_range,
    "validation": segment_count['validation'] * vision_range
    }
learning_rate = 0.5

## Auxiliary functions

### Show result

In [287]:
def show_result(x):
    """Print the result"""
    print("{:.0f}%".format(x * 100))

### Automaticaly generate data sets

## Dataset

In [288]:
low, high = category_min, category_max
dataset = {
    "training": make_samples(sample_count['training'], image_size, low, high),
    "test": make_samples(sample_count['test'], image_size, low, high),
    "validation":make_samples(sample_count['validation'], image_size, low, high)
}

In [289]:
dataset['test']['data'][2].reshape(image_height,image_width)

array([[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [290]:
dataset['test']['labels'][2]

array([ 1.,  0.,  0.])

# Memory allocation

In [291]:
x = tf.placeholder(tf.float32, [None, vision_range])

In [292]:
W = tf.Variable(tf.zeros([vision_range, label_type_count]))

In [293]:
b = tf.Variable(tf.zeros([label_type_count]))

In [294]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [295]:
y_ = tf.placeholder(tf.float32, [None, label_type_count])

In [296]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
)

In [297]:
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

In [298]:
# create a session
session = tf.InteractiveSession()

In [299]:
tf.global_variables_initializer().run()

# I. Regular Neural Net

## Training



In [300]:
session.run(train_step, feed_dict={x: dataset['training']['data'], y_: dataset['training']['labels']})

In [301]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))

In [302]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Test

In [303]:
test_result = session.run(accuracy, feed_dict={x: dataset['test']['data'], y_: dataset['test']['labels']})

In [304]:
show_result(test_result)

100%


## Validation

In [305]:
exam_result = session.run(accuracy, feed_dict={x: dataset['validation']['data'], y_: dataset['validation']['labels']})

In [306]:
show_result(exam_result)

100%
